In [1]:
! unzip /content/drive/MyDrive/Oranges/train.zip

Archive:  /content/drive/MyDrive/Oranges/train.zip
   creating: train/
   creating: train/Blood Oranges_transform/
  inflating: train/Blood Oranges_transform/1.png  
  inflating: train/Blood Oranges_transform/10.png  
  inflating: train/Blood Oranges_transform/11.png  
  inflating: train/Blood Oranges_transform/12.png  
  inflating: train/Blood Oranges_transform/13.png  
  inflating: train/Blood Oranges_transform/14.png  
  inflating: train/Blood Oranges_transform/15.png  
  inflating: train/Blood Oranges_transform/16.png  
  inflating: train/Blood Oranges_transform/17.png  
  inflating: train/Blood Oranges_transform/18.png  
  inflating: train/Blood Oranges_transform/19.png  
  inflating: train/Blood Oranges_transform/2.png  
  inflating: train/Blood Oranges_transform/20.png  
  inflating: train/Blood Oranges_transform/3.png  
  inflating: train/Blood Oranges_transform/4.png  
  inflating: train/Blood Oranges_transform/5.png  
  inflating: train/Blood Oranges_transform/6.png  
  infla

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms
import time
import copy
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
# Define the augmentations and transformations
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly crop and resize the image
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(20),  # Random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation
    transforms.RandomErasing(),  # Random erasing for regularization
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load the dataset
full_dataset = datasets.ImageFolder(root='/content/train', transform=transform_train)

# Split into train and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Apply transforms
train_dataset.dataset.transform = transform_train
val_dataset.dataset.transform = transform_val

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [5]:
from torchvision.models import regnet_y_400mf  # Example model

# Step 1: Define the RegNet model
class RegNetModel(nn.Module):
    def __init__(self, num_classes=5):
        super(RegNetModel, self).__init__()
        # Load pre-trained RegNet-Y 400MF model
        self.model = regnet_y_400mf(pretrained=True)

        # Modify the final layer to match the number of classes in your dataset
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [13]:

# Step 2: Set up the training loop with a condition to stop when validation accuracy exceeds 82%
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=25, patience=5, stop_accuracy=82.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    best_val_accuracy = 0.0
    best_model = None
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

        # Validation phase
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = correct / total
        print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

        # Learning Rate Scheduler step
        scheduler.step(val_loss)

        # Check if the current model is the best
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        # Stop training if validation accuracy exceeds the threshold
        if val_accuracy * 100 >= stop_accuracy:
            print(f'Validation accuracy {val_accuracy * 100:.2f}% exceeded {stop_accuracy}%. Stopping training.')
            break

        ''' # Early stopping
        if patience_counter >= patience:
            print(f'Early stopping after {epoch+1} epochs')
            break'''

    # Load the best model before returning
    model.load_state_dict(best_model)
    return model

In [7]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')
    return accuracy


In [14]:
# Assuming dataloaders is a dictionary with 'train', 'val', and 'test' dataloaders
num_classes = 5  # Change this according to your dataset

# Step 3: Initialize and train the model
model = RegNetModel(num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Assuming train_loader and val_loader are already defined
model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs=100)

Epoch [1/100], Loss: 1.7979
Validation Accuracy: 44.44%
Epoch [2/100], Loss: 0.5558
Validation Accuracy: 50.00%
Epoch [3/100], Loss: 0.2567
Validation Accuracy: 66.67%
Epoch [4/100], Loss: 0.3492
Validation Accuracy: 72.22%
Epoch [5/100], Loss: 0.2835
Validation Accuracy: 66.67%
Epoch [6/100], Loss: 0.2948
Validation Accuracy: 55.56%
Epoch [7/100], Loss: 0.2711
Validation Accuracy: 72.22%
Epoch [8/100], Loss: 0.2606
Validation Accuracy: 66.67%
Epoch [9/100], Loss: 0.1237
Validation Accuracy: 66.67%
Epoch [10/100], Loss: 0.0923
Validation Accuracy: 66.67%
Epoch [11/100], Loss: 0.0572
Validation Accuracy: 61.11%
Epoch [12/100], Loss: 0.0439
Validation Accuracy: 66.67%
Epoch [13/100], Loss: 0.0742
Validation Accuracy: 61.11%
Epoch [14/100], Loss: 0.0643
Validation Accuracy: 61.11%
Epoch [15/100], Loss: 0.1383
Validation Accuracy: 66.67%
Epoch [16/100], Loss: 0.1401
Validation Accuracy: 61.11%
Epoch [17/100], Loss: 0.0555
Validation Accuracy: 61.11%
Epoch [18/100], Loss: 0.0715
Validation 

In [15]:
accuracy = evaluate_model(model,val_loader)

Accuracy on test set: 61.11%


In [16]:
torch.save(model.state_dict(), f'Regnet_best.pth')

In [29]:
model = RegNetModel(num_classes=num_classes) # Initialize model architecture
model.load_state_dict(torch.load('Regnet_best.pth'))  # Load the saved weights
model.eval()
model = model.to(device)

In [34]:
from PIL import Image
transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

image_path  = "/content/Orange-Navel-ZA-DSC_0017-cr-sq-400x400-2.jpg"
image = Image.open(image_path)
image = transform(image).unsqueeze(0)

In [35]:

image = image.to(device)

In [36]:
# Perform inference
with torch.no_grad():
        outputs = model(image)
        print("Model output:", outputs)
        probabilities = torch.softmax(outputs, dim=1)
        print("Class probabilities:", probabilities)
        _, predicted = torch.max(outputs, 1)

    # Map the prediction to the class label
print(predicted.item())
class_names = ['Blood Oranges_transform', 'Navel_transform', 'Tangelo_transform', 'Tangerine_transform', 'cara cara_transform']  # Replace with your actual class names
predicted_class = class_names[predicted.item()]
predicted_class

Model output: tensor([[-0.2909,  1.7062, -0.4555, -1.6438, -0.5715]])
Class probabilities: tensor([[0.0977, 0.7202, 0.0829, 0.0253, 0.0738]])
1


'Navel_transform'